In [1]:
from utils.preprocessing import *
import pandas as pd
from collections import Counter

[nltk_data] Downloading package omw-1.4 to /home/eu3neuom/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
FULL_DATASET_PATH = "./data/dataset.csv"
DATASET_PATH = "./data/"
# run `split_dataset` only once to create train/test/val 
# split_dataset(FULL_DATASET_PATH, DATASET_PATH, verbose=True)

In [ ]:
df = pd.read_csv(os.path.join(DATASET_PATH, "train.csv")).to_numpy()

stats = Counter()
for row in df:
    song = row[5]
    normalized = normalization(song)
    for word in normalized:
        stats.update({word: 1})
print(stats)